In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path

# from fastai.io import *
# from fastai.conv_learner import *
# from fastai.column_data import *

In [2]:
PATH = Path('data/nietzsche')

In [3]:
data = (((PATH/'nietzsche.txt').open()).read())
len(data)

600893

In [4]:
vocab = set(data)

In [5]:
# convert vocab to idx
idxs = list(range(len(vocab)))
idx_to_char = {idx:char for idx,char in enumerate(vocab)}
char_to_idx = {char:idx for idx,char in enumerate(vocab)}

In [6]:
data = [char_to_idx[char] for char in data]
len_data = len(data)

In [7]:
data[:5]

[30, 72, 57, 19, 25]

In [8]:
data = np.array(data)

In [9]:
data = data[:600000]

In [10]:
# a = np.array([1,2,3,4])

# b = np.array([5,6,7,8])

# d = (np.stack((a,b)))

# print(d)
# print(a[0:3])

a = [1,2,3,4]

b = [5,6,7,8]

anp = np.array(a)
bnp = np.array(b)
c = []
c.append(a)
c.append(b)
np.asarray(c)
print(np.stack(c))
d = np.stack((anp,bnp))
print(d)

[[1 2 3 4]
 [5 6 7 8]]
[[1 2 3 4]
 [5 6 7 8]]


In [10]:
# method to create train batches and test batches
def batchwise_data (data, bs = 1, bptt = 3, start=0):
    """
    batchwise_data converts 
    data = 1D np array of indexed chars
    bs - how many examples passed through network at once
    bptt - number of time steps, or length of sequence. 
    start - position at which the sequence of the examples in the first 
        batch begins in the 'data' array 
    """

    len_data = len(data)
    interm_size = int(len_data/bs)
    num_minibatches = interm_size/bptt
    interm_arrays = np.array_split(data,bs) # array (list) of arrays (lists)
#     interm_arrays = np.stack(interm_arrays) # matrix - 1 list per row
#     print(interm_arrays)

    # creating an array of lists of data for each minibatch (x). 
    batches_list = list()

    sequence_starts = list(range(start,len(interm_arrays[0]),bptt))

    for sequence_start in sequence_starts:
        batch_inputs = list()

        for interm in interm_arrays:
            seq = np.array(interm[sequence_start:sequence_start+bptt])
            batch_inputs.append(seq)

        np.asarray(batch_inputs)
        batches_list.append(np.stack(batch_inputs))

#     print(xbatches_list)
#     print(xbatches_list[0])

    return batches_list

In [11]:
# create small dataset so it is easier to debug
smallsz = 2000
datasm = data[:smallsz]

# creating an array of lists of outputs for each minibatch (y). 
xbatches_list = batchwise_data(datasm, bs=2)
ybatches_list = batchwise_data(datasm,bs=2,start=1)

# to drop any remainders of batching
# ybatches_list=ybatches_list[:-1]
# xbatches_list=xbatches_list[:-1]


In [12]:
# apply conditions to make it reproducible:
def align(xbatches, ybatches):
    """
    both inputs as a list of np arrays
    """
    lenx = len(xbatches)
    leny = len(ybatches)
    if lenx>leny:
        for i in range(lenx-leny):
            xbatches = xbatches[:-1]
    
    elif leny>lenx:
        for i in range(leny-lenx):
            ybatches = ybatches[:-1]
            
    
    return xbatches,ybatches

In [13]:
print(len(xbatches_list))
print(len(ybatches_list))

334
333


In [14]:
xbatches_list,ybatches_list = align(xbatches_list,ybatches_list)

In [15]:
print(xbatches_list[-2:])
print(ybatches_list[-2:])

[array([[ 5, 43, 63],
       [56, 14, 40]]), array([[32, 46, 82],
       [63, 66, 77]])]
[array([[43, 63, 32],
       [14, 40, 63]]), array([[46, 82, 71],
       [66, 77, 14]])]


In [16]:
len_vocab = len(vocab)

In [17]:
from sklearn.preprocessing import OneHotEncoder

def intlabel_ohencoding(array):
    
    """
    purpose of this method is to convert array's values
    into onehot encoded values based on the number of 
    categories present exclusively in array
    
    array - must be integer label encoded already
    
    returns a list of onehot encoded data examples to be 
        reshaped into original sizes ie 
        (dim1,dim2,dim3,...,dim(n-1),-1)
    """
    # data already in integers so don't need sklearn LabelEncoder, just OneHotEncoder
    shape = array.shape
    flat_size=1
    for i in shape:
        flat_size*=i

    arr_flat = array.reshape((flat_size),1)

    # Create an encoder instance, fit on data, and transform int labels
    enc = OneHotEncoder()
    enc.fit(arr_flat)
    onehotlabels_arrflat = enc.transform(arr_flat).toarray()
    
    # reshape back to original with additional dimension for encoded array
    onehotlabels = np.reshape(onehotlabels_arrflat,(*array.shape,-1))
    return onehotlabels

In [18]:
# converting y-values to one hot encodings 
"""
using sklearn's onehotencoder for its efficiencies
"""

ybatches_arr = np.stack(ybatches_list)
ybatches_onehot = intlabel_ohencoding(ybatches_arr)
ybatches_onehot=np.reshape(ybatches_onehot,(*ybatches_arr.shape,-1))
# ybatches_onehot[0][3]

In [35]:
# three char model therefore need lists of every first, second, third and fourth char. 
op = 3
xt1 = list(np.array(data)[range(0,len(data),op+1)])
xt2 = list(np.array(data)[range(1,len(data),op+1)])
xt3 = list(np.array(data)[range(2,len(data),op+1)])
y = list(np.array(data)[range(3,len(data),op+1)])

### Vanilla RNN Model (Numpy) Pipeline

##### Input data form - old

In [13]:
op, xt1[:5], len(xt1), xt2[:5], len(xt2), xt3[:5], len(xt3), y[:5], len(y)

(3,
 [11, 46, 22, 11, 62],
 150224,
 [74, 17, 22, 11, 9],
 150223,
 [14, 14, 19, 36, 31],
 150223,
 [53, 22, 33, 19, 77],
 150223)

In [14]:
data[:10], len(data)

([11, 74, 14, 53, 46, 17, 14, 22, 22, 22], 600893)

In [27]:
bptt = 3 # the number of time steps, since output of 3rd time step is the prediction. 
x_train=np.stack([xt1[1:],xt2,xt3],1)
y_train=y

In [42]:
bs = 300
len(np.array_split(x_train,int(len(x_train)/bs)))

500

In [29]:

x_train.shape

(150223, 3)

##### Input data form - updated

In [19]:
# Will have to loop over to access char for a certian time step.
x = np.asarray(xbatches_list)
y = ybatches_onehot
print(x.shape,y.shape)

(333, 2, 3) (333, 2, 3, 52)


##### Model initiation

In [21]:
def softmax(array):
    # assumes batch dimension is the first
    arr_exp = np.exp(array)
    sum_example = np.sum(arr_exp, axis = 1)
    return arr_exp/sum_example[:1]
    
# test def
#softmax(np.random.randint(-10,10,(10,5)))



In [ ]:
class NumpyRNN():
    
    def __init__(self, x, y, nhidden, bs, nvocab): # forget nfac for embeddings, for simplicity.
        self.h = np.zeros(bs,nhidden)
        self.U = np.random.uniform(low=-0.01,high=0.01,size=((nhidden+1),nhidden)) #why nhidden+1?
        self.V = np.random.uniform(low=-0.01,high=0.01,size=(nhidden,nvocab))
        self.W = np.eye(nhidden)
    
    def forward(self,x):
        
        """
        x assumed to be array of sequential inputs for one bptt - size: bs x bptt. Each time
        step's input should therefore have bs rows and 1 col. 
        Form in which x is passed is assumed to be 3 columns vectors, each for each time step
        
        """
        bptt = len(x) # will give number of time-steps
        o = [] # list to store output at each time step
        hs = [self.h] # list to store hidden state after each time step 
        
        for i in range(bptt):
            x_in = x[:,i]
            cat_in = np.concatenate((self.h,x_in),-1) # resultant size = bs x (nhidden+1)
            cat_in_U = np.tanh(np.dot(cat_in,self.U)) # bs x nhidden
            # should ideally have relu here, but for simplicity, not include
            self.h = np.tanh(np.dot(cat_in_U, W)) # bs x nhidden
            hs.append(self.h)      
            out_presoft = np.tanh(np.dot(self.h,self.V)) # bs x nvocab      
            #do softmax
            out = softmax(out_presoft)    
            o.append(out)

        return o, h
    
        
    

In [ ]:
def bptt(self, x, y_hat): #y_hat is the actual/true value of prediction; size - bs x 1
    
    #CHECK ALL OPS' DIMENSIONS
    
    dV = np.zeros((nhidden,nvocab)) # dLdV
    dU = np.zeros((nhidden+1),nhidden) # dLdU
    dW = np.zeros((nhidden,nhidden)) # dLdW
    
    o, h = self.forward(x)
    T = len(h) #number of time steps
    
    dV__dV = 1
    dV_m_tanh__dV = (np.tanh(np.tanh(np.dot(h[-1],self.W)))
    dtanh__dV_m_tanh = (1-np.dot(dV_m_tanh__dV,self.V)**2)
    Si = softmax(np.tanh(np.dot(self.V,dV_m_tanh__dV))) # bs x nvocab
    Sj = softmax((np.dot(self.V,dV_m_tanh__dV))) # bs x nvocab
    dout__doutpresoft = Si #will need a 3 dimensional array
    
    dE__dV = (1-(self.V*(np.tanh(np.tanh(self.h))))**2)*(np.tanh(np.tanh(self.W*self.h)))
    
    # softmax der https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/
    
    
    return dU,dV,dW
    
    
NumpyRNN.bptt = bptt

In [ ]:
model = Numpy

#### Model Specs
* Ignored embeddings for simplicity - taking letters as indices. 

### Vanilla RNN Model - PyTorch - incomplete
Will implement the following characteristics: 
* Concatenation of previous time-step's hidden state and next input
* Instantiate all weight matrices (U, V, W) using random numbers initially - change to identity matrix later 
* 

In [ ]:
class ThreeCharRNN(nn.Module):
    def __init__(self, nvocab, nfac, nhidden):
        super().__init__()
        
        self.e = nn.Embedding(nvocab, nfac)
        self.e.weight.data.uniform_(-0.01,0.01) # must actually use kaiming he initialization guidelines. 
        
        self.lin_in = nn.Linear(nhidden,(nhidden+nfac))
        self.lin_in.weight.data.uniform_(-0.01,0.01)
        self.lin_hidden = nn.Linear(nhidden,nhidden)
        self.lin_hidden.weight.data.uniform_(-0.01,0.01)
        self.lin_out = nn.Linear(nhidden,nvocab)
        self.lin_out.weight.data.uniform_(-0.01,0.01)
        
    def forward(self,*cs):
        
        h = torch.zeroes(bs,nhidden)
        for c in cs:
            xin = self.e(c)
            cat_in = torch.cat(h,xin,dim=1)
            h = F.tanh(self.lin_hidden(self.lin_in(cat_in))) # or ReLU??
        
        return F.log_softmax(self.lin_out(h))
            
        
        
        
        

###### Need to do to above class still
* consolidate dimensions
* ensure initialization is actually needed
* chekc computation of non-linear functions

In [58]:
val_idx = get_cv_idxs(len(idxs)-op-1)

In [ ]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

### GRU PyTorch Workflow - incomplete